# LangChain: Q&A over Documents
An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
#pip install --upgrade langchain

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [ ]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [ ]:
#!pip install docarray
#!pip install dashscope

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.llms import Tongyi

In [ ]:
embedding = DashScopeEmbeddings(
    dashscope_api_key=os.environ.get("OPENAI_API_KEY"),
    model="text-embedding-v1"
)

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding,
).from_loaders([loader])

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
llm = Tongyi(temperature=0,
             dashscope_api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
display(Markdown(response))

In [ ]:
loader = CSVLoader(file_path=file)

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

In [ ]:
embedding = DashScopeEmbeddings(
    dashscope_api_key=os.environ.get("OPENAI_API_KEY"),
    model="text-embedding-v1"
)

In [ ]:
embed = embedding.embed_query("Hi my name is Harrison")

In [ ]:
print(len(embed))

In [ ]:
print(embed[:5])

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embedding
)

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
len(docs)


In [ ]:
docs[0]


In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature=0.0,
                 model="qwen-plus",
                 api_key=os.environ.get("OPENAI_API_KEY"),
                 base_url=os.environ.get("OPENAI_API_URL"))

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)


In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
display(Markdown(response))

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding,
).from_loaders([loader])